In [8]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle

# Demographic Profile Data

In [6]:
# boston_pop = pd.read_csv ('Boston/bigquery_BUA_population.csv', index_col = False)
# boston_pop['person_id'] = boston_pop['person_id'].astype(float)
# # only keep the columns we need
# boston_pop = boston_pop[['person_id','age','individual_income','household_size','family_structure','vehicles']]
# boston_pop.head(3)

,person_id,age,individual_income,household_size,family_structure,vehicles
0,1.208496e+19,88,178836,2_person,nonfamily_single,1
1,1.707506e+19,88,54048,3_person,married_couple,1
2,3.803337e+18,88,84751,2_person,married_couple,1


# Trip Data

In [7]:
# ##Import trip info
# trips_sa = pd.read_csv ('Boston/BUA_trips_bga_sa_amenitiesadded2.csv', index_col = False)
# trips_th = pd.read_csv ('Boston/BUA_trips_bga_th_amenitiesadded2.csv', index_col = False)
# trips_th.head(2)

,activity_id,person_id,household_id,mode,duration_minutes,distance_miles,origin_bgrp,destination_bgrp,naics_category,placekey,naics_code,location_name,top_category,sub_category,vehicles,sub_amenity,naics_4,travel_purpose,top_amenity
0,5.171354e+18,7.734123e+18,1.163440e+19,ON_DEMAND_AUTO,6.0,1.493868,2.502506e+11,2.502508e+11,72,zzw-222@62j-sjp-qj9,722320.0,M & M Ribs,Special Food Services,Caterers,3,Special Food Services,7223,Eat,F&B Eatery
1,1.678649e+19,1.748523e+18,1.581380e+19,CARPOOL,17.0,5.193408,2.502504e+11,2.502508e+11,72,22b-226@62j-sgf-j9z,722515.0,Caffè Nero,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,2,Snack and Nonalcoholic Beverage Bars,7225,Eat,F&B Eatery


In [8]:
# cond1 = {
#     'BIKING': 'Biking',
#     'CARPOOL':'Car',
#     'ON_DEMAND_AUTO':'Car',
#     'PRIVATE_AUTO':'Car',
#     'PUBLIC_TRANSIT':'Public_transport',
#     'WALKING':'Walking'
# }

# trips_sa['mode'] = trips_sa['mode'].map(cond1)
# trips_th['mode'] = trips_th['mode'].map(cond1)
# trips_sa=trips_sa[trips_sa['mode'].notnull()]
# trips_th=trips_th[trips_th['mode'].notnull()]
# trips_sa.sort_index(axis = 1)
# trips_th.sort_index(axis = 1)

# # only keep the columns we need
# trips_sa = trips_sa[['activity_id','person_id','travel_purpose','top_amenity','sub_amenity','top_category','sub_category','mode','distance_miles','duration_minutes','location_name']]
# trips_th  = trips_th[['activity_id','person_id','travel_purpose','top_amenity','sub_amenity','top_category','sub_category','mode','distance_miles','duration_minutes','location_name']]
# trips_sa.head(2)

,activity_id,person_id,travel_purpose,top_amenity,sub_amenity,top_category,sub_category,mode,distance_miles,duration_minutes,location_name
0,1.415185e+19,3.820938e+18,Recreation,Leisure & Wellness,Gym,Other Amusement and Recreation Industries,Fitness and Recreational Sports Centers,Car,101.180729,111.0,Balance At Urbanity Dance
1,5.174423e+18,1.503045e+19,Recreation,Entertainment,Zoos and Nature Parks,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,Public_transport,7.325792,59.0,Franklin Park


In [9]:
# trips_th_with_pop = trips_th.merge(boston_pop, how = 'inner', on = 'person_id')
# trips_th_with_pop.head(2)

,activity_id,person_id,travel_purpose,top_amenity,sub_amenity,top_category,sub_category,mode,distance_miles,duration_minutes,location_name,age,individual_income,household_size,family_structure,vehicles
0,5.171354e+18,7.734123e+18,Eat,F&B Eatery,Special Food Services,Special Food Services,Caterers,Car,1.493868,6.0,M & M Ribs,80,6060,7_plus_person,family_single,3_plus
1,1.678649e+19,1.748523e+18,Eat,F&B Eatery,Snack and Nonalcoholic Beverage Bars,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,Car,5.193408,17.0,Caffè Nero,29,87673,2_person,nonfamily_single,2


In [10]:
# trips_th_with_pop.to_csv('data/orig/trips_th_with_pop.csv',index = False)

# Create Behavior Graph 

Nodes: 
- Person
    - person_id
    - age_group
    - income_group
    - household_size
    - family_structure
    - vehicles

- Desire
    - travel_purpose
        - Eat
        - Recreation
        - Shop
        - Errands
        - Other
        - School
        - Work

- Intension
    - top_amenity/top_category
    - sub_amenity/sub_category
    - mode
    - distance_miles
    - duration_minutes
    - location


Links:

Person ---want_to---> Desire ---choose_to---> Intension

In [11]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle

In [12]:
trips_th_with_pop = pd.read_csv('../data/orig/trips_th_with_pop.csv',index_col = False)
trips_th_with_pop = trips_th_with_pop[trips_th_with_pop['travel_purpose'] == 'Eat']
# shuffle the data
trips_th_with_pop  = trips_th_with_pop.sample(frac=1).reset_index(drop=True)


In [13]:
# remove the rows where family structure equals to 'GQ'
trips_th_with_pop = trips_th_with_pop[trips_th_with_pop['family_structure']!='GQ']
# remove the rows where vehicles equals to 'GQ'
trips_th_with_pop = trips_th_with_pop[trips_th_with_pop['vehicles']!='GQ']
trips_th_with_pop['vehicles'] = trips_th_with_pop['vehicles'].replace('zero',0)
trips_th_with_pop.head(2)

,activity_id,person_id,travel_purpose,top_amenity,sub_amenity,top_category,sub_category,mode,distance_miles,duration_minutes,location_name,age,individual_income,household_size,family_structure,vehicles
1,1.823868e+19,6.498351e+18,Eat,F&B Eatery,Limited-Service Restaurants,Restaurants and Other Eating Places,Limited-Service Restaurants,Public_transport,2.875662,20.0,Potbelly Sandwich Works,65,0,4_person,married_couple,1
2,1.109140e+19,5.784824e+18,Eat,F&B Eatery,Snack and Nonalcoholic Beverage Bars,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,Walking,1.573514,30.0,Dunkin',20,14046,6_person,married_couple,3_plus


In [4]:
person_props = ['age','individual_income','household_size','family_structure','vehicles']

desire_prop = ['travel_purpose']

intension_prop = ['top_amenity','sub_amenity','mode','distance_miles','duration_minutes','location_name']

# Prepare train data

In [5]:
person_df = pd.DataFrame(columns = ['id','age','individual_income','household_size','family_structure','vehicles', 'description'])
desire_df = pd.DataFrame(columns = ['id', 'desire', 'description'])
intention_df = pd.DataFrame(columns = ['id','target_amenity','mode','distance_miles','duration_minutes','location_name', 'description'])

want_to_edge_df = pd.DataFrame(columns = ['source','target','type'])
go_to_edge_df = pd.DataFrame(columns = ['source','target','type'])

for idx,row in trips_th_with_pop.iterrows():
    # if idx < 10000:
    #     continue
    person_id = row['person_id']
    person_description = f"A {row['age']} year old person, living in a {row['family_structure']} family with {row['household_size']} members. The person has {row['vehicles']} vehicles and an annual income of {row['individual_income']} dollars."
    
    desire = row['travel_purpose']
    desire_description = f"This person wants to {row['travel_purpose']}"


    intention_id = row['activity_id']
    intention_description = f"This person goes to {row['location_name']} by {row['mode']}, this place is in the amenity category of {row['top_amenity']}/{row['sub_amenity']}. The distance is {row['distance_miles']} miles. It takes {row['duration_minutes']} minutes."


    person_df.loc[len(person_df)] = [f"Person_{person_id}",row['age'],row['individual_income'],row['household_size'],row['family_structure'],row['vehicles'],person_description]

    desire_df.loc[len(desire_df)] = [f"Desire_{person_id}_{desire}",row['travel_purpose'],desire_description]

    intention_df.loc[len(intention_df)] = [f"Internsion_{intention_id}",row['top_amenity']+'/'+row['sub_amenity'],row['mode'],row
    ['distance_miles'],row['duration_minutes'],row['location_name'],intention_description]

    want_to_edge_df.loc[len(want_to_edge_df)] = [f"Person_{person_id}",f"Desire_{person_id}_{desire}",'want_to']

    go_to_edge_df.loc[len(go_to_edge_df)] = [f"Desire_{person_id}_{desire}",f"Internsion_{intention_id}",'go_to']

    if idx>1000:
        break

In [6]:
person_df = person_df.drop_duplicates()
desire_df = desire_df.drop_duplicates()
intention_df = intention_df.drop_duplicates()
want_to_edge_df = want_to_edge_df.drop_duplicates()
go_to_edge_df = go_to_edge_df.drop_duplicates()
print("number of person nodes: ",len(person_df))
print("number of desire nodes: ",len(desire_df))
print("number of intension nodes: ",len(intention_df))
print("number of want_to edges: ",len(want_to_edge_df))
print("number of go_to edges: ",len(go_to_edge_df))

number of person nodes:  863
number of desire nodes:  864
number of intension nodes:  866


In [8]:
dataset = 'train'
person_df.to_csv(f'data/{dataset}/1K/person.csv',index = False)
desire_df.to_csv(f'data/{dataset}/1K/desire.csv',index = False)
intention_df.to_csv(f'data/{dataset}/1K/intention.csv',index = False)
want_to_edge_df.to_csv(f'data/{dataset}/1K/want_to_edge.csv',index = False)
go_to_edge_df.to_csv(f'data/{dataset}/1K/go_to_edge.csv',index = False)

# Prepare test data

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle

In [3]:
trips_th_with_pop = pd.read_csv('../data/orig/trips_th_with_pop.csv',index_col = False)
# shuffle the data
trips_th_with_pop  = trips_th_with_pop.sample(frac=1).reset_index(drop=True)
# remove the rows where family structure equals to 'GQ'
trips_th_with_pop = trips_th_with_pop[trips_th_with_pop['family_structure']!='GQ']
# remove the rows where vehicles equals to 'GQ'
trips_th_with_pop = trips_th_with_pop[trips_th_with_pop['vehicles']!='GQ']
trips_th_with_pop['vehicles'] = trips_th_with_pop['vehicles'].replace('zero',0)
trips_th_with_pop.head(2)


,activity_id,person_id,travel_purpose,top_amenity,sub_amenity,top_category,sub_category,mode,distance_miles,duration_minutes,location_name,age,individual_income,household_size,family_structure,vehicles
0,7.118115e+18,1.179076e+19,Other,Transport Hub,Bus Station,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,Car,2.490252,8.0,MBTA State Street Station,45,1054,2_person,married_couple,2
1,8.288944e+18,8.767809e+17,Eat,F&B Eatery,Full-Service Restaurants,Restaurants and Other Eating Places,Full-Service Restaurants,Public_transport,1.429528,13.0,Beantown Burrito,48,101082,6_person,married_couple,2


In [4]:
income_bins = [-float('inf'), 0, 30000, 60000, 90000, 120000, float('inf')]
income_labels = ['Debt', 'Low', 'Moderate', 'High', 'Very High', 'Ultra High']
age_bins = [-float('inf'), 18, 30, 40, 50, 60, float('inf')]
age_labels = ['Teen', 'Young Adult', 'Adult', 'Middle Age', 'Senior', 'Elderly']

trips_th_with_pop['income_group'] = pd.cut(trips_th_with_pop['individual_income'], bins=income_bins, labels=income_labels)
trips_th_with_pop['age_group'] = pd.cut(trips_th_with_pop['age'], bins=age_bins, labels=age_labels)
trips_th_with_pop.head(2)

,activity_id,person_id,travel_purpose,top_amenity,sub_amenity,top_category,sub_category,mode,distance_miles,duration_minutes,location_name,age,individual_income,household_size,family_structure,vehicles,income_group,age_group
0,7.118115e+18,1.179076e+19,Other,Transport Hub,Bus Station,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,Car,2.490252,8.0,MBTA State Street Station,45,1054,2_person,married_couple,2,Low,Middle Age
1,8.288944e+18,8.767809e+17,Eat,F&B Eatery,Full-Service Restaurants,Restaurants and Other Eating Places,Full-Service Restaurants,Public_transport,1.429528,13.0,Beantown Burrito,48,101082,6_person,married_couple,2,Very High,Middle Age


In [5]:
duration_step = 5
distance_step = 0.5

In [6]:
trips_th_with_pop['target_amenity'] = trips_th_with_pop['top_amenity']+'/'+trips_th_with_pop['sub_amenity']
trips_th_with_pop['distance_miles'] = trips_th_with_pop['distance_miles']//distance_step*distance_step
trips_th_with_pop['duration_minutes'] = trips_th_with_pop['duration_minutes']//duration_step*duration_step

In [7]:
trips_th_with_pop = trips_th_with_pop[['person_id','travel_purpose','target_amenity','mode',
'distance_miles','duration_minutes','age','individual_income','household_size','family_structure','vehicles',
'income_group','age_group']]

trips_th_with_pop.head()

,person_id,travel_purpose,target_amenity,mode,distance_miles,duration_minutes,age,individual_income,household_size,family_structure,vehicles,income_group,age_group
0,1.179076e+19,Other,Transport Hub/Bus Station,Car,2.0,5.0,45,1054,2_person,married_couple,2,Low,Middle Age
1,8.767809e+17,Eat,F&B Eatery/Full-Service Restaurants,Public_transport,1.0,10.0,48,101082,6_person,married_couple,2,Very High,Middle Age
2,1.814368e+19,Eat,F&B Eatery/Full-Service Restaurants,Car,2.0,5.0,42,55557,7_plus_person,nonfamily_single,3_plus,Moderate,Middle Age
3,1.241317e+19,Recreation,Hotel/Hotel and Lodging,Car,3.5,5.0,42,40791,6_person,married_couple,3_plus,Moderate,Middle Age
4,1.294959e+19,Eat,F&B Eatery/Full-Service Restaurants,Car,3.0,10.0,27,22674,7_plus_person,nonfamily_single,3_plus,Low,Young Adult


In [8]:
for idx,row in trips_th_with_pop.iterrows():
    person_description = f"A {row['age']} year old person, living in a {row['family_structure']} family with {row['household_size']} members. The person has {row['vehicles']} vehicles and an annual income of {row['individual_income']} dollars."
    trips_th_with_pop.loc[idx,'description'] = person_description
    cypher_profile = f"A {row['age_group']} person, whose income is {row['income_group']}"
    trips_th_with_pop.loc[idx,'cypher_profile'] = cypher_profile

trips_th_with_pop.head()



,person_id,travel_purpose,target_amenity,mode,distance_miles,duration_minutes,age,individual_income,household_size,family_structure,vehicles,income_group,age_group,description,cypher_profile
0,1.179076e+19,Other,Transport Hub/Bus Station,Car,2.0,5.0,45,1054,2_person,married_couple,2,Low,Middle Age,"A 45 year old person, living in a married_coup...","A Middle Age person, whose income is Low"
1,8.767809e+17,Eat,F&B Eatery/Full-Service Restaurants,Public_transport,1.0,10.0,48,101082,6_person,married_couple,2,Very High,Middle Age,"A 48 year old person, living in a married_coup...","A Middle Age person, whose income is Very High"
2,1.814368e+19,Eat,F&B Eatery/Full-Service Restaurants,Car,2.0,5.0,42,55557,7_plus_person,nonfamily_single,3_plus,Moderate,Middle Age,"A 42 year old person, living in a nonfamily_si...","A Middle Age person, whose income is Moderate"
3,1.241317e+19,Recreation,Hotel/Hotel and Lodging,Car,3.5,5.0,42,40791,6_person,married_couple,3_plus,Moderate,Middle Age,"A 42 year old person, living in a married_coup...","A Middle Age person, whose income is Moderate"
4,1.294959e+19,Eat,F&B Eatery/Full-Service Restaurants,Car,3.0,10.0,27,22674,7_plus_person,nonfamily_single,3_plus,Low,Young Adult,"A 27 year old person, living in a nonfamily_si...","A Young Adult person, whose income is Low"


In [9]:
trips_th_with_pop.loc[0,'cypher_profile']

'A Middle Age person, whose income is Low'